In [9]:
from ultralytics import YOLO
import cv2
import winsound
import time
import requests
import tkinter as tk
from tkinter import messagebox
from threading import Thread
import threading


In [13]:
# 🔹 LINE Notify Setup 
LINE_NOTIFY_TOKEN = "wDWc4yjjTH90dosYSXKva3tfhzW5Qz38NHgGHgt8HfL"
LINE_NOTIFY_URL = "https://notify-api.line.me/api/notify"

In [14]:


# Function to send LINE Notify message with image
def send_line_notify(message, image_path=None):
    headers = {"Authorization": f"Bearer {LINE_NOTIFY_TOKEN}"}
    data = {"message": message}

    if image_path:
        files = {"imageFile": open(image_path, "rb")}
        requests.post(LINE_NOTIFY_URL, headers=headers, data=data, files=files)
        files["imageFile"].close()
    else:
        requests.post(LINE_NOTIFY_URL, headers=headers, data=data)

# Function to show pop-up alert and stop further notifications
def show_popup(message, stop_alert_event):
    root = tk.Tk()
    root.withdraw()
    response = messagebox.askyesno("🔥 Fire Alert!", message)

    if response:
        print("User confirmed fire alert!")
        stop_alert_event.set()  # Stop further alerts
    else:
        print("User ignored fire alert!")
    root.destroy()

# Load trained YOLO model
model = YOLO(r"runs/detect/train2/weights/best.pt")

# Open video file
video_path = "test_video/2110972-uhd_3840_2160_30fps.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Failed to load video!")
    exit()

fire_detected = False
last_alert_time = 0
fire_detected_time = 0
last_fire_location = None

# Create an event to stop alerts
stop_alert_event = threading.Event()

# Get frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit when the video ends

    # Run YOLO fire detection
    results = model(frame)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            label = result.names[int(box.cls[0])]

            # If fire is detected with confidence > 50%
            if "fire" in label.lower() and conf > 0.50:
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                box_size = (x2 - x1) * (y2 - y1)  # Calculate bounding box size

                # **Determine horizontal position**
                if center_x < frame_width * 0.33:
                    horizontal_pos = "Left side"
                elif center_x > frame_width * 0.66:
                    horizontal_pos = "Right side"
                else:
                    horizontal_pos = "Center"

                # **Determine vertical position**
                if center_y < frame_height * 0.33:
                    vertical_pos = "Top"
                elif center_y > frame_height * 0.66:
                    vertical_pos = "Bottom"
                else:
                    vertical_pos = "Middle"

                # **Determine depth (Near-Far)**
                if box_size > (frame_width * frame_height * 0.05):  # If bounding box is large
                    depth_pos = "Near the camera"
                elif box_size < (frame_width * frame_height * 0.02):  # If bounding box is small
                    depth_pos = "Far from the camera"
                else:
                    depth_pos = "Mid-range"

                fire_location = f"{horizontal_pos}, {vertical_pos}, {depth_pos}"
                print(f"🚨 Fire detected at: {fire_location}")

                # Draw bounding box and display position
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, f"{fire_location}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

                # Save the frame with fire detection
                img_filename = "fire_detected_frame.jpg"
                cv2.imwrite(img_filename, frame)

                # Check if fire has been detected in the same location for more than 30 seconds
                current_time = time.time()
                if fire_location != last_fire_location:
                    fire_detected_time = current_time  # Reset timer when fire is detected in a new location
                    last_fire_location = fire_location

                if current_time - fire_detected_time < 30:  # If fire has been detected for less than 30 seconds
                    if current_time - last_alert_time > 10:  # Alert every 10 seconds
                        if not stop_alert_event.is_set():  # Only alert if stop signal is not set
                            # Send alerts to LINE Notify with the image attached
                            alert_message = f"🔥 Fire detected at: {fire_location}"

                            # Show pop-up alert in a separate thread
                            Thread(target=show_popup, args=(alert_message, stop_alert_event)).start()

                            # Send alerts to LINE Notify with the image attached
                            Thread(target=send_line_notify, args=(alert_message, img_filename)).start()

                            last_alert_time = current_time

                else:
                    # If fire has been detected for more than 30 seconds, stop sending alerts
                    print("Fire alert stopped for this location.")

    # Resize frame for display
    frame_resized = cv2.resize(frame, (1280, 720))
    cv2.imshow("🔥 Fire Detection System", frame_resized)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Clean up by deleting the saved image file
if os.path.exists(img_filename):
    os.remove(img_filename)



0: 384x640 (no detections), 44.0ms
Speed: 3.0ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.2ms
Speed: 4.0ms preprocess, 55.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.0ms
Speed: 3.0ms preprocess, 49.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 50.0ms
Speed: 4.0ms preprocess, 50.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.0ms
Speed: 3.0ms preprocess, 37.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 43.0ms
Speed: 3.0ms preprocess, 43.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 43.0ms
Speed: 4.0ms preprocess, 43.0ms i

Exception in thread Thread-49 (show_popup):
Traceback (most recent call last):
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13460\972653500.py", line 17, in show_popup
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\tkinter\messagebox.py", line 114, in askyesno
    s = _show(title, message, QUESTION, YESNO, **options)
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\tkinter\messagebox.py", line 76, in _show
    res = Message(**options).show()
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\tkinter\commondialog.py", line 45, in show
    s = master.tk.call(self.command, *master._options(self.options))
Run

0: 384x640 2 fires, 43.0ms
Speed: 3.0ms preprocess, 43.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Bottom, Far from the camera

0: 384x640 2 fires, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Bottom, Far from the camera

0: 384x640 3 fires, 44.0ms
Speed: 3.0ms preprocess, 44.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Middle, Far from the camera

0: 384x640 4 fires, 43.0ms
Speed: 3.0ms preprocess, 43.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Middle, Far from the camera

0: 384x640 2 fires, 43.0ms
Speed: 5.0ms preprocess, 43.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Middle, Far from the camera

0: 384x640 2 fires, 43.0ms
Speed: 5.0ms preprocess, 43.0ms inference, 2.0ms postprocess per ima

Exception in thread Thread-51 (show_popup):
Traceback (most recent call last):
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13460\972653500.py", line 17, in show_popup
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\tkinter\messagebox.py", line 114, in askyesno
    s = _show(title, message, QUESTION, YESNO, **options)
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\tkinter\messagebox.py", line 76, in _show
    res = Message(**options).show()
  File "c:\Users\ADMIN\anaconda3\envs\dl_env\lib\tkinter\commondialog.py", line 45, in show
    s = master.tk.call(self.command, *master._options(self.options))
Run

0: 384x640 1 fire, 43.3ms
Speed: 3.0ms preprocess, 43.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Middle, Far from the camera

0: 384x640 2 fires, 44.0ms
Speed: 4.0ms preprocess, 44.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Middle, Far from the camera

0: 384x640 2 fires, 45.0ms
Speed: 3.0ms preprocess, 45.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Middle, Far from the camera

0: 384x640 2 fires, 46.0ms
Speed: 3.0ms preprocess, 46.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Middle, Far from the camera

0: 384x640 2 fires, 45.0ms
Speed: 3.0ms preprocess, 45.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
🚨 Fire detected at: Center, Bottom, Far from the camera

0: 384x640 2 fires, 47.0ms
Speed: 3.0ms preprocess, 47.0ms inference, 2.0ms postprocess per imag

NameError: name 'os' is not defined